In [1]:
import xml.etree.ElementTree as ET

In [2]:
raw_data = ET.parse('../scenario/vehroute-output.xml')
data = {}

In [6]:
def parse_route(point, route):
    if 'exitTimes' in route.attrib:
        point['edges'] = [edge for edge in route.attrib['edges'].split()]
        point['exitTimes'] = [int(float(time)) for time in route.attrib['exitTimes'].split()]
    return point

root = raw_data.getroot()

for veh in root:
    point = {}
    point['type'] = veh.attrib['type']
    point['depart'] = int(float(veh.attrib['depart']))
    point['arrival'] = int(float(veh.attrib['arrival']))
    for r in veh:
        if r.tag != 'route':
            for route in r:
                point = parse_route(point, route)
        else:
            point = parse_route(point, r)
    data[veh.attrib['id']] = point
                    

In [8]:
import json
json.dumps('vehroutes.json')

'"vehroutes.json"'

In [10]:
with open('vehroutes.json', 'w') as f:
    json.dump(data, f)

In [1]:
#for veh in data:
#    print("{} : {}, {}, {}, {}, {}".format(veh, data[veh]['edges'], data[veh]['exitTimes'], data[veh]['depart'], data[veh]['arrival'], data[veh]['type']))
    

### Network analysis

In [3]:
import sumolib
net = sumolib.net.readNet('../scenario/lust.net.xml')

In [4]:
edges = net.getEdges()
print(len(edges))

5779


In [6]:
edgeHighways = [edge for edge in edges if edge.getLaneNumber() > 2]

In [7]:
len(edgeHighways)

823

In [3]:
import json
lambdas_file = json.load(open('lambdas.json'))

In [4]:
total_lambda = 0
max_lambda = 0
min_lambda = float("inf")
for edge in lambdas_file:
    total_lambda += lambdas_file[edge]
    if lambdas_file[edge] > max_lambda:
        max_lambda = lambdas_file[edge]
    if lambdas_file[edge] < min_lambda:
        min_lambda = lambdas_file[edge]

In [5]:
print(total_lambda/len(lambdas_file))
print(max_lambda)
print(min_lambda)


46.26245443499393
62
36


In [11]:
total_lambda * 3

114222

## Counting number of trips per time step for plot

In [1]:
from xml.etree import ElementTree as ET

In [2]:
tree1 = ET.parse('../scenario/DUARoutes/local.0.rou.xml')
tree2 = ET.parse('../scenario/DUARoutes/local.1.rou.xml')
tree3 = ET.parse('../scenario/DUARoutes/local.2.rou.xml')

In [3]:
root1 = tree1.getroot()
root2 = tree2.getroot()
root3 = tree3.getroot()

In [4]:
trips = [0 for i in range(24*3600)]

In [5]:
for veh in root1:
    trips[int(float(veh.attrib['depart']))] += 1

In [6]:
for veh in root2:
    trips[int(float(veh.attrib['depart']))] += 1
for veh in root3:
    trips[int(float(veh.attrib['depart']))] += 1

In [7]:
total_trips = 0
for t in trips:
    total_trips += trips[t]
print(total_trips)

40886


In [15]:
 %matplotlib qt

In [34]:
import matplotlib.pyplot as plt
import numpy as np

In [20]:
plt.plot([i for i in range(len(trips))], trips)
plt.show()

In [29]:
trips_avg = []
for i, trip in enumerate(trips[200:-200]):
    trips_avg.append(800*sum(trips[i-200:i+200])/400)
    

In [32]:
len(trips_avg)

86000

In [42]:
yaxis = np.linspace(0,24, len(trips_avg))
fontsize = 42
plt.plot(yaxis, trips_avg)
plt.grid(True)
plt.xlabel('Time of Day (seconds)', fontsize=fontsize)
plt.ylabel('Running Vehicles', fontsize=fontsize)
plt.xticks([i for i in range(25) if i%2 == 0],fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.show()

## Calculating vulnerability ranking of full simulation

In [43]:
from xml.etree import ElementTree as ET

In [44]:
tree = ET.parse('../output/vehroutes.xml')

In [45]:
root = tree.getroot()

In [46]:
nominal_data = {}
for vehicle in root:
    nominal_data[vehicle.attrib['id']] = {'start': int(float(vehicle.attrib['depart'])),
                                         'end': int(float(vehicle.attrib['arrival']))}

In [48]:
travel_time_nom = {}
for v in nominal_data:
    travel_time_nom[v] = nominal_data[v]['end'] - nominal_data[v]['start']

In [52]:
denom = sum(travel_time_nom.values())